# 最近傍点の探索（単純な方法）
点群$X$から任意の点（クエリとも呼びます）$\bf p$の最近傍点を見つけ出すタスクのことを最近傍点を探索や単に検索と呼びます．
点間の距離として二乗距離を使うことにすると，このタスクは以下の式で定式化することができます．

\begin{equation}
\label{eq:dist_point-pcd}
d({\bf p},X) = \min_{{\bf x} \in X}|| {\bf p} - {\bf x} ||
\end{equation}

最もシンプルな方法は，$X$を構成する点全てと$\bf p$との二乗距離を計算し，
その最小値を結果とする方法です．
この方法は，$X$が$m$点で構成される点群とすると，計算量は$O(m)$です．
Open3Dを使ってこの処理を実装してみましょう．

In [ ]:
import open3d as o3d
import numpy as np
import copy

$X$として，sin関数にしたがう点列を用意します．
点${\bf p} = (1.0,0.0,0.0)$としました．

In [ ]:
# point cloud as sin function
X_x = np.arange(-np.pi,np.pi, 0.1)
X_y = np.sin(X_x)
X_z = np.zeros(X_x.shape)
X = np.vstack([X_x, X_y, X_z]).T

In [ ]:
# point p
p = np.array([1.0,0.0,0.0])

In [ ]:
# open3d point cloud of X
pcd_X = o3d.geometry.PointCloud()
pcd_X.points = o3d.utility.Vector3dVector(X)
pcd_X.paint_uniform_color([0.5,0.5,0.5])

# open3d point cloud of p
pcd_p = o3d.geometry.PointCloud()
pcd_p.points = o3d.utility.Vector3dVector([p])
pcd_p.paint_uniform_color([0.0,0.0,1.0])

# Create axis
mesh = o3d.geometry.TriangleMesh.create_coordinate_frame()

# Visualization
o3d.visualization.draw_geometries([mesh, pcd_X,pcd_p])

点と点群の距離を計算する関数は```dist(p,X)```です．
$\bf p$の最近傍点までの二乗距離とその点のインデクスを返却します．
この関数を使って，点${\bf p}$の最近傍点の色を緑に変更して可視化しましょう．

In [ ]:
def dist( p, X ):
    dists = np.linalg.norm(p-X,axis=1) 
    min_dist = min(dists)
    min_idx = np.argmin(dists)
    
    return min_dist, min_idx

In [ ]:
min_dist, min_idx = dist(p,X)
np.asarray(pcd_X.colors)[min_idx] = [0.0,1.0,0.0]
print("distance:{}, idx:{}".format(min_dist, min_idx))
o3d.visualization.draw_geometries([mesh,pcd_X,pcd_p])

# Open3Dによるkd-tree
それでは，Open3Dを使ってkd-treeによる探索を試してみましょう．Open3Dにおいても，前節までの説明と同様に，まずkd-treeを構築し，クエリ点を入力することによって探索を行います．

また，Open3Dのkd-treeには，探索の基準が3つ用意されています．
- ```search_knn_vector_3d``` ... クエリのk近傍点を抽出する方法
- ```search_radius_vector_3d``` ... 指定した半径の値以内の点を抽出する方法
- ```search_hybrid_vector_3d``` ... 上記２つの基準を満たす点を抽出する方法．RKNNサーチとも呼ばれます．

それぞれ動作を確認しましょう．

まずは，```search_knn_vector_3d```を試します．
点群の10000番目の点をクエリとし，そこから近い順で200点を抽出します．

In [ ]:
# Load a point cloud and paint it gray.
pcd = o3d.io.read_point_cloud("../data/bun000.pcd")
pcd.paint_uniform_color([0.5, 0.5, 0.5])

# build td-tree
pcd_tree = o3d.geometry.KDTreeFlann(pcd)

query = 10000
pcd.colors[query] = [1, 0, 0]

[k, idx, d] = pcd_tree.search_knn_vector_3d(pcd.points[query], 200)
np.asarray(pcd.colors)[idx[1:], :] = [0, 0, 1]
o3d.visualization.draw_geometries([pcd])

dには各点の二乗距離が入っているので，ルートを取ると，実際の距離になります．

つぎに，```search_radius_vector_3d```を試します．
点群の20000番目の点をクエリとして，そこから距離0.01以内の点を抽出します．

In [ ]:
query = 20000
pcd.colors[query] = [1, 0, 0]
[k, idx, d] = pcd_tree.search_radius_vector_3d(pcd.points[query], 0.01)
np.asarray(pcd.colors)[idx[1:], :] = [0, 1, 0]
o3d.visualization.draw_geometries([pcd])

最後に，```search_hybrid_vector_3d```を試します．
点群の5000番目の点をクエリとして，そこから距離0.01以内の点を200点抽出します．

In [ ]:
query = 5000
pcd.colors[query] = [1, 0, 0]
[k, idx, d] = pcd_tree.search_hybrid_vector_3d(pcd.points[query], 
                                               radius=0.01,
                                               max_nn=200)
np.asarray(pcd.colors)[idx[1:], :] = [0, 1, 1]
o3d.visualization.draw_geometries([pcd])